## Upload kaggle.json file
This allows access to Kaggle datasets from Colab.

In [ ]:
from google.colab import files
files.upload()

## Set up Kaggle API key
We move `kaggle.json` to the correct directory and give it proper permissions.

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

## Download dataset from Kaggle

In [ ]:
!kaggle datasets download -d rjmanoj/credit-card-customer-churn-prediction

## Unzip dataset

In [ ]:
!unzip credit-card-customer-churn-prediction.zip

## Load dataset using Pandas

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("Churn_Modelling.csv")
df.head()

## Dataset Information

In [ ]:
df.info()

## Check for duplicates

In [ ]:
df.duplicated().sum()

## Drop irrelevant columns
We remove `RowNumber`, `CustomerId`, and `Surname`. They don't help in prediction.

In [ ]:
df.drop(columns=['RowNumber','CustomerId','Surname'], inplace=True)
df.head(3)

## One-hot encode categorical columns
We convert `Geography` and `Gender` into numerical columns.

In [ ]:
df = pd.get_dummies(df, columns=['Geography','Gender'], drop_first=True)
df.head(3)

## Split data into features (X) and target (y)

In [ ]:
x = df.drop(columns='Exited')
y = df['Exited']

## Split dataset into training and testing sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=42)

## Standardize features
We use StandardScaler to normalize data.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

## Build ANN Model

In [ ]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(8, activation='relu', input_dim=11))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

## Compile model

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

## Train model

In [ ]:
hist = model.fit(X_train_scale, Y_train, epochs=100, validation_split=0.2)

## Plot training and validation loss

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history['loss'], label='Training Loss')
plt.plot(hist.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

## Make predictions

In [ ]:
y_pred = np.where(model.predict(X_test_scale) > 0.5, 1, 0)
y_pred[:10]

## Evaluate model performance

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, y_pred)

## Predict on a sample customer

In [ ]:
sample_customer = {
    "CreditScore": 600,
    "Age": 40,
    "Tenure": 3,
    "Balance": 60000,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 50000,
    "Geography_Germany": 0,
    "Geography_Spain": 1,
    "Gender_Male": 1
}
sample_df = pd.DataFrame([sample_customer])
sample_df = sample_df[X_train.columns]
sample_scaled = scaler.transform(sample_df)
prediction = model.predict(sample_scaled)
threshold = 0.3
result = "Exit (Churn)" if prediction[0][0] > threshold else "Stay"

print("Prediction Probability:", prediction[0][0])
print("Final Prediction:", result)